# Lab 1.2: Conversational RAG - Mortgage Policy Assistant

In this lab, we will build a **Banking Policy Assistant** for Wells Fargo. This assistant allows loan officers to query internal mortgage guidelines (e.g., Conforming vs. Jumbo loans) and ask follow-up questions while maintaining conversational context.

## Use Case
A loan officer needs to quickly check credit score requirements for different loan types without searching through a 200-page PDF manual. They might ask:
1. "What is the min credit score for a Jumbo loan?"
2. "How does that compare to FHA?" (Contextual follow-up)

## Key Concepts
1. **History Aware Retriever**: Rephrases the user's latest query using the chat history so it makes sense as a standalone query to the vector store.
2. **Chat History**: Maintaining line of conversation.

In [ ]:
# 1. Install Dependencies
print("Installing dependencies...")
%pip install -qU langchain langchain-groq langchain-community langchain-huggingface chromadb sentence-transformers
print("Dependencies installed.")

In [ ]:
# 2. Setup API Keys
import getpass
import os

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

In [ ]:
# 3. Setup Vector Store (Mortgage Policy Data)
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

# Load Internal Policy Document
print("Loading mortgage policy document...")
loader = TextLoader("./data/mortgage_policy.txt")
docs = loader.load()
print(f"Loaded {len(docs)} document(s).")

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splits = text_splitter.split_documents(docs)
print(f"Split into {len(splits)} chunks.")

# Embed & Store
print("Creating vector store...")
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)
retriever = vectorstore.as_retriever()
print("Vector store created.")

## 4. History Aware Retriever
We need a chain that takes the `chat_history` and the `input` and generates a search query.

In [ ]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Initialize LLM
llm = ChatGroq(
    model="qwen/qwen3-32b",
    temperature=0,
    reasoning_format="parsed"
)

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# Chain to rephrase question
contextualize_q_chain = contextualize_q_prompt | llm | StrOutputParser()
print("Contextualization chain created.")

In [ ]:
# Let's test the contextualization to understand what it does
from langchain_core.messages import HumanMessage, AIMessage

print("--- Testing Query Contextualization ---")
# Mock history: user asked about Jumbo loans, AI answered.
sample_history = [
    HumanMessage(content="What is the min credit score for a Jumbo Loan?"),
    AIMessage(content="The minimum credit score is 700.")
]
# User asks a follow-up specific to the context (referring to 'that')
sample_input = "How does that compare to FHA?"
print(f"Chat History: {len(sample_history)} messages")
print(f"User Follow-up: {sample_input}")

rephrased_query = contextualize_q_chain.invoke({"chat_history": sample_history, "input": sample_input})
print(f"Rephrased Query (for Vector Store): {rephrased_query}")
print("---------------------------------------")

## 5. QA Chain with History
Now we create the final chain that uses the retrieved documents to answer.

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa_system_prompt = (
    "You are a specialized Mortgage Policy Assistant for Wells Fargo. "
    "Use the following pieces of retrieved policy context to answer "
    "the loan officer's question. If you don't know the answer, say that you "
    "cannot find it in the policy. Keep answers professional and concise."
    "\n\n"
    "{context}"
)

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

def contextualized_question(input: dict):
    if input.get("chat_history"):
        return contextualize_q_chain
    else:
        return input.get("input")

rag_chain = (
    RunnablePassthrough.assign(
        context=contextualized_question | retriever | format_docs
    )
    | qa_prompt
    | llm
    | StrOutputParser()
)
print("Full RAG chain created.")

## 6. Testing the Chat
We can now manage specific chat sessions.

In [ ]:
chat_history = []

# First Question: Specific Policy Query
user_input = "What is the minimum credit score for a Jumbo Loan?"

print(f"User: {user_input}")
# rag_chain returns string now
answer = rag_chain.invoke({"input": user_input, "chat_history": chat_history})
print(f"AI: {answer}")

# Update History
chat_history.extend([HumanMessage(content=user_input), AIMessage(content=answer)])

# Second Question (Follow-up): Contextual Comparison
print("\n--- Follow up ---")
user_input = "How does that compare to FHA loans?"
print(f"User: {user_input}")

answer = rag_chain.invoke({"input": user_input, "chat_history": chat_history})
print(f"AI: {answer}")

chat_history.extend([HumanMessage(content=user_input), AIMessage(content=answer)])